In [1]:
from tqdm import tqdm
import numpy as np
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from scipy.spatial import distance
from heapq import nsmallest
from sklearn.metrics import pairwise_distances
import random

In [2]:
articles = pd.read_csv('articlesDataset/cleanDataArticles', delimiter=',', header=None)
tables = pd.read_csv('articlesDataset/cleanDataTablesFile', delimiter=',', header=None)

In [3]:
formattedTables = tables.iloc[:,:].values
formattedArticles = articles.iloc[:,:].values

In [4]:
corpusTables = []

for tables in formattedTables:
    
    corpusTables.append(str(tables[6]))

In [48]:
def selectRandomArticles():
    
    for i in range(10):
    
        randomArticle = random.choice(formattedArticles)
    
        corpusArticlesID.append(randomArticle[0])
        corpusArticlesText.append(str(randomArticle[1]))

In [6]:
vectorizer = TfidfVectorizer(stop_words='english', analyzer='word')
tablesByCharacteristics = vectorizer.fit_transform(corpusTables)

In [8]:
def getTotalTablesByArticle(idArticle):

    countTopTables = 0

    for tables in formattedTables:
    
        if tables[0] == int(idArticle):
        
             countTopTables = countTopTables + 1
    
    return countTopTables

In [9]:
def getIdRankedTalbes(topK,distanceVector):

    idRankedTables = []

    for topkDistance in topK:
        
        index = np.where(distanceVector == topkDistance)
         
        indexColummun = index[0][0]
        
        idRankedTables.append(formattedTables[indexColummun][0])

    return idRankedTables

In [34]:
def getAccuracy(idRankedTables, idQueryGoal):

    accuracy = 0

    for idTable in idRankedTables:
    
        if idTable == idQueryGoal:
    
            accuracy = 1
            break;

    return accuracy

In [78]:
AverageTop5 = []
AverageTop10 = []
AverageTop15 = []
AverageTop20 = []

for execution in tqdm(range(30)):
    
    print("execution = "+ str(execution))
    
    corpusArticlesID = []
    corpusArticlesText = []
    
    selectRandomArticles()
    
    #print(corpusArticlesID)
    
    articlesByCharacteristics = vectorizer.transform(corpusArticlesText)
    
    for accuracyK in range(5,21,5):
    
        accuracy = []

        totalSelectedArticles = int(articlesByCharacteristics.shape[0])

        for i in range(totalSelectedArticles):
    
            distanceVector = pairwise_distances(articlesByCharacteristics[i], tablesByCharacteristics, metric='cosine')
    
            idQueryGoal = int(corpusArticlesID[i])
    
            countTopTables = accuracyK
        
            topKRank = nsmallest(countTopTables, distanceVector[0])
    
            idRankedTables = getIdRankedTalbes(topKRank,distanceVector[0])
    
            accuracy.append(getAccuracy(idRankedTables,idQueryGoal))
    
        if accuracyK == 5:
            
            AverageTop5.append(np.mean(accuracy))
        
        if accuracyK == 10:
            
            AverageTop10.append(np.mean(accuracy))
        
        if accuracyK == 15:
            
            AverageTop15.append(np.mean(accuracy))
        
        if accuracyK == 20:
            
            AverageTop20.append(np.mean(accuracy))
            
        #print("TOP "+str(accuracyK)+" = "+str(np.mean(accuracy)))

  0%|          | 0/30 [00:00<?, ?it/s]

execution = 0


  3%|▎         | 1/30 [00:12<05:51, 12.10s/it]

execution = 1


  7%|▋         | 2/30 [00:24<05:39, 12.13s/it]

...

 93%|█████████▎| 28/30 [05:50<00:24, 12.27s/it]

execution = 28


 97%|█████████▋| 29/30 [06:02<00:12, 12.22s/it]

execution = 29


100%|██████████| 30/30 [06:14<00:00, 12.15s/it]


In [83]:
np.std(AverageTop5)

0.09451631252505219

In [86]:
np.std(AverageTop10)

0.1175679472569893

In [87]:
np.std(AverageTop15)

0.1303414319734477

In [88]:
np.std(AverageTop20)

0.13337499349161705